# 基于MindSpore框架的MASS案例实现
## 1 模型简介
微软亚洲研究院于2019在ICML发表《MASS: Masked Sequence to Sequence Pre-training for Language Generation》，其借鑑了Bert的Masked Language Model预训练任务，提出了MAsked Sequence to Sequence Pre-training（MASS）模型，为自然语言生成任务联合预训练编码器和解码器。

MASS的编码器-解码器结构示例，图中“_”表示被屏蔽的词。

<div align=center>
<img src='https://i.imgur.com/Jvhm0Dx.png' width='600px'>
</div>

编码器： 以被随机屏蔽掉连续片段的句子作为输入，BERT的做法是随机屏蔽掉15%的词，而MASS为了解决编码与解码之间的平衡，做法为屏蔽掉句子总长50%的片段。模型中使用特殊符号 $[\mathbb M]$ 替换连续的单词来屏蔽片段，起始位置是随机的，且被选中的token有80%的概率是正常的 $[\mathbb M]$ token，10%的概率是被随机token替换，10%的概率保持原来的token。以上图为例，其中输入序列有8个单词，片段 $x_3-x_6$ 被屏蔽掉。

解码器：输入为与编码器同样的序列，但是会屏蔽掉剩馀的词，然后解码器只预测编码器端屏蔽掉的词。以上图为例，只给定 $x_3x_4x_5$ 作为位置 4 - 6 的解码器输入，解码器会将  $[\mathbb M]$ 作为其他位置的输入（屏蔽了位置 1 − 3 和 7 − 8）。为了减少内存和计算成本，被屏蔽的token会被移除，未屏蔽token的位置编码不变（如果前两个标记被屏蔽并移除，第三个标记的位置仍然是 2 而不是 0)。通过这种方式，可以获得相似的准确度，并在解码器中减少 50% 的计算量。

```
encoder input (source): [x1, x2, x3, x4, x5, x6, x7, x8, </eos>]
masked encoder input:   [x1, x2, x3,  _,  _,  _, x7, x8, </eos>]
decoder input:          [  -, x3, x4, x5]
                          |   |   |   |
                          V   V   V   V
decoder output:         [x3, x4, x5, x6]
```

MASS预训练有以下几大优势：

(1) 编码器被强制去抽取未被屏蔽掉的词的含义，可以提升编码器理解源序列文本的能力。<br>
(2) 通过在解码器端预测连续的标记，解码器可以比仅预测离散标记拥有更好的语言建模能力。<br>
(3) 通过在解码器端进一步屏蔽在编码器端未被屏蔽掉的词， 以鼓励解码器从编码器端提取更多有用的信息来做预测，而不是依赖于前面预测出的单词，这样能促进编码器-解码器结构的联合训练。

### 1.1 模型结构

其模型基础结构可以使用任何Seq2Seq的结构，由于Transformer的优越性，故论文中使用Transformer模型作为基础结构，Transformer整体架构由编码器和解码器两个部分组成，不依赖任何RNN和CNN结构来生成输出，而是使用了Attention注意力机制，自动学习输入序列中每个单词和其他单词的关联，可以更好的处理长文本，且该模型可以高效的并行工作，训练速度较快。

Transformer 的整体架构如下：

<div align=center>
<img src='https://i.imgur.com/ooO7ULP.png' width='400px'>
</div>

- 编码器和解码器分别由 $N=6$ 个相同的编码器/解码器层组成。
- 在 Transformer 架构的左半部分，编码器的任务是将输入序列映射到一系列连续表示，然后将其馈送到解码器。
- 架构右半部分的解码器接收编码器的输出以及前一个时间步的解码器输出，以生成输出序列。
- 解码器的输出最终通过一个全连接层，然后是一个 softmax 层，以生成对输出序列下一个单词的预测。

Transformer的运作过程举例如下图，这是一个由法文翻译为英文的 Transformer，从编码器端输入一句法文“je suis etudiant”，从解码器端则输出翻译出的英文“I am a student”。

<div align=center>
<img src='https://i.imgur.com/GQKvNAd.png' width='350'/>
</div>

1. 整句法文“je suis etudiant”做为编码器的输入，编码器一次处理整个句子，产生输出 $X$。
2. 解码器以 $X$ 及空序列做为其输入，产生第一个输出 I。
3. 解码器以 $X$ 及 I 做为其输入，产生第二个输出 am。
4. 解码器以 $X$ 及 I am 做为其输入，产生第三个输出 a。
5. 解码器以 $X$ 及 I am a 做为其输入，产生第四个输出 student。
6. 解码器以 $X$ 及 I am a student 做为其输入，产生第五个输出 <end of sentence>。


### 1.2目标函数

给定一个未配对的源句子 $x ∈ \mathcal X$ ，MASS通过被屏蔽的序列 $x^{\setminus u:v}$ 作为输入来预测句子片段 $x^{u:v}$ 以预训练序列到序列模型。目标函数为一极大似然函数：

$$
L(\theta; \mathcal X) = \frac{1}{|\mathcal X|} \sum_{x \in \mathcal X} \log P(x^{u:v} | x^{\setminus u:v}; \theta) \\ = \frac{1}{|\mathcal X|} \log \Pi^{v}_{t=u} P(x^{u:v}_{t}|x^{u:v}_{\textless t}, x^{\setminus u:v};\theta)
$$

注: $x^{u:v}$ 表示以句子位置 $u$ 为起点 $v$ 为终点的片段； $x^{\setminus u:v}$ 为 $x^{u:v}$ 的修改版本，从 $u$ 到 $v$ 的片段被屏蔽， $0 < u < v < m$ 其中 $m$ 是句子 $x$ 长度。

### 1.3 模型特点
MASS 有一个重要的超参数 $k$，表示屏蔽的连续片段长度，通过调整 $k$ 的大小，MASS 能包含 BERT 中的掩码语言模型训练方法以及 GPT 中标准的语言模型预训练方法，使 MASS 成为一个通用的预训练框架。

当 $k = 1$ 时，根据MASS的设定，编码器端仅屏蔽一个单词，解码器以源序列中未屏蔽的单词为条件预测这个单词，如图(a)所示。由于解码器的所有输入都被屏蔽了，因此解码器本身就像一个非线性分类器，类似于 BERT 中使用的 softmax 矩阵。在这种情况下，条件概率为 $P (x^u|x^{\setminus u}; θ)$， $u$ 是掩码标记的位置，这正是 BERT3中使用的掩码语言模型的公式。

当 $k = m$（ $m$ 为序列长度）时，根据MASS的设定，编码器会屏蔽所有的单词，解码器需要预测所有单词，如图(b)所示。由于编码器端所有词都被屏蔽了，解码器的注意力机制相当于没有获取到信息，在这种情况下条件概率为 $P(x^{1:m}|x^{\setminus 1:m}; θ)$，等价于GPT中的标准语言模型。

![](https://i.imgur.com/JdTGb9z.png)


## 2 模型构建
以下我们使用代码来建构Transformer模型。

### 2.1 Embedding and Positional Encoding

编码器与解码器在输入前的资料处理模块，包括 Input Embedding （Output Embedding) 和 Positional Encoding。

<div align=center><img src='https://i.imgur.com/4OVf1jD.png' width='400px'/></div>

在处理文本之前，需要先对其进行分词(tokenizer)，将文本切分成一个个子字串(token)，每个字串有相对完整的语意，便于后续将token转换为嵌入(Embedding)和位置编码(Positional Encoding)的向量表达。其中 Embedding 可以理解成一个词汇表(lookup table)，有了词汇表后就可以通过查表的方式得到每一个词的词向量。EmbeddingLookup类的主要功能就是创建词汇表，它的核心代码其实只有一行：
```python
self.embedding_table = Parameter(normal_weight([vocab_size, embedding_size], embedding_size))
```
词汇表（embedding_table）是一个Parameter类型，说明它里面的值是通过不断迭代学习得到的，效果是使相似的词的词向量有很高的相似度。它的shape为[vocal_size, embed_dim]，其中vocal_size代表所有词汇的总个数，是经过BPE编码得到的；embed_dim是指每一个词用多少维度的向量表示，本案例中将它的值设置为1024。

In [1]:
import numpy as np
import mindspore.common.dtype as mstype
from mindspore import nn
from mindspore.ops import operations as P
from mindspore.common.tensor import Tensor
from mindspore.common.parameter import Parameter

class EmbeddingLookup(nn.Cell):
    def __init__(self,
                 vocab_size,
                 embed_dim):
        super(EmbeddingLookup, self).__init__()
        self.embedding_dim = embed_dim
        self.vocab_size = vocab_size

        init_weight = np.random.normal(0, embed_dim ** -0.5, size=[vocab_size, embed_dim]).astype(np.float32)
        # 0 is Padding index, thus init it as 0.
        init_weight[0, :] = 0
        # 核心代码
        self.embedding_table = Parameter(Tensor(init_weight))
        
        self.expand = P.ExpandDims()
        self.gather = P.Gather()
        self.one_hot = P.OneHot()
        self.on_value = Tensor(1.0, mstype.float32)
        self.off_value = Tensor(0.0, mstype.float32)
        self.array_mul = P.MatMul()
        self.reshape = P.Reshape()
        self.get_shape = P.Shape()

    def construct(self, input_ids):
        # input_ids (Tensor): A batch of sentences with shape (N, T).
        _shape = self.get_shape(input_ids)  # (N, T).
        _batch_size = _shape[0]
        _max_len = _shape[1]

        flat_ids = self.reshape(input_ids, (_batch_size * _max_len,))
        output_for_reshape = self.gather(self.embedding_table, flat_ids, 0)

        output = self.reshape(output_for_reshape, (_batch_size, _max_len, self.embedding_dim))
        # Returns: Tensor, word embeddings with shape (N, T, D)
        return output, self.embedding_table

前面提到，Transformr不包含RNN和CNN结构，而是使用Attention 机制，其忽略了输入序列中每一个元素的位置及元素之间的距离，这样更容易找出序列中每一个元素的关连性，这是它的优点。然而在实际 NLP 应用中，句子内单词的顺序有着相当大的语法及语义上的含义，是不能忽略的。
Transformer为了弥补这一缺点，添加了一个“位置编码 (Positional Encoding)”机制：输入序列在经过 embedding 处理后，每一个位置的输入值，都要加上该位置的位置编码，如此便可以在输入序列中加入位置信息。

Position Encoding可以使用训练学习，也可以使用固定公式计算。Transformer采用的是不同频率的正弦和余弦函数：
$$
PE_{pos,2i} = sin(pos/10000^{2i/d_{\mathrm{model}}}) \\
PE_{pos,2i+1} = cos(pos/10000^{2i/d_{\mathrm{model}}})
$$
其中，$pos$ 指的是这个 word 在这个句子中的位置，$i$ 指的是 embedding 维度($i ∈ {0, 1, 2, 3, …, d_{\mathrm model}}$，它在偶数位置使用sin函数编码，在奇数位置使用cos函数编码)，$d_{\mathrm model}$ 为模型的 embedding 大小。最后将其与Word Embedding相加后得到的结果就是编码器或解码器的输入。

In [2]:
def position_encoding(length, depth,
                      min_timescale=1,
                      max_timescale=1e4):
    """Create Tensor of sinusoids of different frequencies."""
    depth = depth // 2
    positions = np.arange(length, dtype=np.float32)
    log_timescale_increment = (np.log(max_timescale / min_timescale) / (depth - 1))
    inv_timescales = min_timescale * np.exp(
        np.arange(depth, dtype=np.float32) * -log_timescale_increment)
    scaled_time = np.expand_dims(positions, 1) * np.expand_dims(inv_timescales, 0)
    x = np.concatenate([np.sin(scaled_time), np.cos(scaled_time)], axis=1)
    return x


class PositionalEmbedding(nn.Cell):
    """Add positional info to word embeddings."""
    def __init__(self,
                 embedding_size,
                 max_position_embeddings=512):
        super(PositionalEmbedding, self).__init__()
        self.add = P.Add()
        self.expand_dims = P.ExpandDims()
        self.position_embedding_table = Tensor(
            position_encoding(max_position_embeddings, embedding_size),
            mstype.float32
        )
        self.gather = P.Gather()
        self.get_shape = P.Shape()

    def construct(self, word_embeddings):
        input_shape = self.get_shape(word_embeddings)
        input_len = input_shape[1]
        # 核心代码
        position_embeddings = self.position_embedding_table[0:input_len:1, ::]
        position_embeddings = self.expand_dims(position_embeddings, 0)
        output = self.add(word_embeddings, position_embeddings)
        return output

### 2.2 注意力机制

Transformer 所用的 Attention 机制，是使用基于「Self-Attention」的「Scaled Dot-Product Attention」，并在同一个子层中，使用 8 个平行的 「Scaled Dot-Product Attention」，成为「Multi-Head」Attention 结构。

<div align=center>
   <img src='https://i.imgur.com/99YUvcZ.png'/ width=500>
</div>

输入序列Q/K/V，对于编码器的第一层而言，它们是经过 Embedding 和位置编码的的输入序列，对于编码器的第二层以上的各层而言，它们是前一层的输出。

在编码器中，计算单头注意力的方式如下：<br>
输入 $[x_1, x_2, x_3, x_4...]$(其中 $x_i$是一个Word Embedding)，每一个 Scaled Dot-Product Attention 有一组属于它自己的权重矩阵 ($W_Q, W_K, W_V$)，输入 $x_i$ 和 $W_Q$ 作矩阵相乘，即得 $q_i$，以此类推，算出各个输入对应的 $q/k/v$，设置Query, key, Value三个矩阵。

<div align=center>
   <img src='https://i.imgur.com/AF6zqj5.png'/ width=500>
</div>

透过这三个矩阵计算注意力分数，执行步骤为：<br>
第一步 (MatMul)：将 Q 和序列中所有的 K 作内积 (Dot-Product) ，得到一个内积序列。

- 例如 $q_1$与 $k_1, k_2, k_3, k_4$ 做点积运算，则输出为 $[a_{1,1}, a_{1,2}, a_{1,3}, a_{1,4}]$， $a_{i, j}$ 表示第 $j$ 个Word Embedding对第 $i$ 个的重要性。

第二步 (Scale)：将内积序列中的每个数除以「Ｋ长度的平方根」。<br>
第三步 (SoftMax)：将序列做 SoftMax 运算。<br>
第四步 (MatMul)：将输入 V 序列中的值，乘上其对应的 SoftMax 结果，得到一个 weighted V 序列。<br>
第五步 (Sum)：将 weighted V 序列所有元素相加，得到输出的结果。

#### 2.2.1 MultiHeadAttention

多头自注意力的计算方式与单头自注意力相同，只是 $W_q, W_k, W_v$ 会重复head_num次。

本案例中 $W_q = W_k = W_v =$ attn_embed_dim // num_attn_heads，attn_embed_dim设置为1024，num_attn_heads设置为8。

In [3]:
class MultiHeadAttention(nn.Cell):
    def __init__(self,
                 src_dim,
                 tgt_dim,
                 attn_embed_dim,
                 num_attn_heads=1,
                 query_act=None,
                 key_act=None,
                 value_act=None,
                 out_act=None,
                 has_attention_mask=True,
                 attention_dropout_prob=0.0,
                 initializer_range=0.02,
                 do_return_2d_tensor=True,
                 compute_type=mstype.float32):
        super(MultiHeadAttention, self).__init__()

        self.attn_embed_dim = attn_embed_dim
        self.num_attn_heads = num_attn_heads
        self.size_per_head = attn_embed_dim // num_attn_heads
        self.src_dim = src_dim
        self.tgt_dim = tgt_dim
        self.has_attention_mask = has_attention_mask
        
        self.scores_mul = Tensor([1.0 / math.sqrt(float(self.size_per_head))],
                                 dtype=compute_type)
        self.reshape = P.Reshape()
        # 首先设置Q、K、V矩阵，它们的值是通过不断迭代学习到的, shape为[units, from_tensor_width]。代码中设置它的shape为[1024, 64]。
        self.query_layer = nn.Dense(src_dim,
                                    attn_embed_dim,
                                    activation=query_act,
                                    has_bias=True,
                                    weight_init=TruncatedNormal(initializer_range)).to_float(compute_type)
        self.key_layer = nn.Dense(tgt_dim,
                                  attn_embed_dim,
                                  activation=key_act,
                                  has_bias=True,
                                  weight_init=TruncatedNormal(initializer_range)).to_float(compute_type)
        self.value_layer = nn.Dense(tgt_dim,
                                    attn_embed_dim,
                                    activation=value_act,
                                    has_bias=True,
                                    weight_init=TruncatedNormal(initializer_range)).to_float(compute_type)

        self.out_layer = nn.Dense(attn_embed_dim,
                                  attn_embed_dim,
                                  activation=out_act,
                                  has_bias=True,
                                  weight_init=TruncatedNormal(initializer_range)).to_float(compute_type)

        self.matmul_trans_b = P.BatchMatMul(transpose_b=True)
        self.multiply = P.Mul()
        self.transpose = P.Transpose()
        self.multiply_data = Tensor([-10000.0], dtype=compute_type)
        self.matmul = P.BatchMatMul()

        self.softmax = nn.Softmax()
        self.dropout = nn.Dropout(1.0 - attention_dropout_prob)

        if self.has_attention_mask:
            self.expand_dims = P.ExpandDims()
            self.sub = P.Sub()
            self.add = P.Add()
            self.cast = P.Cast()
            self.get_dtype = P.DType()

        self.do_return_2d_tensor = do_return_2d_tensor
        self.cast_compute_type = SaturateCast(dst_type=compute_type)
        self.softmax_cast = P.Cast()
        self.get_shape = P.Shape()
        self.transpose_orders = (0, 2, 1, 3)

    def construct(self, queries, keys, values, attention_mask):
        q_shape = self.get_shape(queries)  # (N, T, D)
        batch_size = q_shape[0]
        src_max_len = q_shape[1]

        k_shape = self.get_shape(keys)  # (N, T', D)
        tgt_max_len = k_shape[1]

        _src_4d_shape = (batch_size, src_max_len, self.num_attn_heads, self.size_per_head)
        _tgt_4d_shape = (batch_size, tgt_max_len, self.num_attn_heads, self.size_per_head)

        queries_2d = self.reshape(queries, (-1, self.src_dim))
        keys_2d = self.reshape(keys, (-1, self.tgt_dim))
        values_2d = self.reshape(values, (-1, self.tgt_dim))

        query_out = self.query_layer(queries_2d)  # (N*T, D)*(D, D) -> (N*T, D)
        key_out = self.key_layer(keys_2d)  # (N*T, D)*(D, D) -> (N*T, D)
        value_out = self.value_layer(values_2d)  # (N*T, D)*(D, D) -> (N*T, D)

        query_out = self.multiply(query_out, self.scores_mul)

        query_layer = self.reshape(query_out, _src_4d_shape)
        query_layer = self.transpose(query_layer, self.transpose_orders)  # (N, h, T, D')
        key_layer = self.reshape(key_out, _tgt_4d_shape)
        key_layer = self.transpose(key_layer, self.transpose_orders)  # (N, h, T', D')
        value_layer = self.reshape(value_out, _tgt_4d_shape)
        value_layer = self.transpose(value_layer, self.transpose_orders)  # (N, h, T', D')

        attention_scores = self.matmul_trans_b(query_layer, key_layer)
        # 编码器中不需要做Mask，而解码器中每一个子层皆要执行
        if self.has_attention_mask:
            attention_mask = self.expand_dims(attention_mask, 1)
            multiply_out = self.sub(
                self.cast(F.tuple_to_array((1.0,)), self.get_dtype(attention_scores)),
                self.cast(attention_mask, self.get_dtype(attention_scores))
            )  # make mask position into 1, unmask position into 0.
            adder = self.multiply(multiply_out, self.multiply_data)
            adder = self.softmax_cast(adder, mstype.float32)
            attention_scores = self.softmax_cast(attention_scores, mstype.float32)
            attention_scores = self.add(adder, attention_scores)

        attention_scores = self.softmax_cast(attention_scores, mstype.float32)
        attention_prob = self.softmax(attention_scores)
        attention_prob = self.softmax_cast(attention_prob, self.get_dtype(key_layer))
        attention_prob = self.dropout(attention_prob)
        
        # (N, h, T, T')*(N, h, T', D') -> (N, h, T, D')
        context_layer = self.matmul(attention_prob, value_layer)
        context_layer = self.transpose(context_layer, self.transpose_orders)  # (N, T, h, D')
        context_layer = self.reshape(context_layer,
                                     (batch_size * src_max_len, self.attn_embed_dim))  # (N*T, D)

        context_layer = self.out_layer(context_layer)

        if not self.do_return_2d_tensor:
            context_layer = self.reshape(
                context_layer, (batch_size, src_max_len, self.attn_embed_dim)
            )  # (N, T, D)

        return context_layer

#### 2.2.2 SelfAttention
SelfAttention类中按顺序调用MultiheadAttention类、LayerNorm类(Layer Norm)、ResidualConnection类（Redual Layer)。<br>
对应网络框架图中的 MultiheadAttention -> Add & Norm。

<div align=center>
    <img src='https://i.imgur.com/5KQFzmm.png' width='500px'>
</div>

In [4]:
class SelfAttention(nn.Cell):
    def __init__(self,
                 attn_embed_dim,
                 num_attn_heads,
                 attn_dropout_prob=0.1,
                 initializer_range=0.02,
                 dropout_prob=0.1,
                 has_attention_mask=True,
                 compute_type=mstype.float32):
        super(SelfAttention, self).__init__()
        self.multi_head_self_attention = MultiHeadAttention(
            src_dim=attn_embed_dim,
            tgt_dim=attn_embed_dim,
            attn_embed_dim=attn_embed_dim,
            num_attn_heads=num_attn_heads,
            attention_dropout_prob=attn_dropout_prob,
            initializer_range=initializer_range,
            has_attention_mask=has_attention_mask,
            do_return_2d_tensor=False,
            compute_type=compute_type)

        self.layer_norm = LayerNorm(in_channels=attn_embed_dim)
        self.residual = ResidualConnection(dropout_prob=dropout_prob)

    def construct(self, queries, keys, values, attention_mask):
        q = self.layer_norm(queries)  # (N, T, D)
        attention_output = self.multi_head_self_attention(
            q, keys, values, attention_mask
        )  # (N, T, D)
        q = self.residual(attention_output, queries)
        return q

#### 2.2.3 LayerNorm
LayerNorm类实现了Layer Normalization方法，它可以有效的促进网络学习。

In [5]:
class LayerNorm(nn.Cell):

    def __init__(self, in_channels=None, return_2d=False):
        super(LayerNorm, self).__init__()
        self.return_2d = return_2d
        self.layer_norm = nn.LayerNorm((in_channels,))
        self.cast = P.Cast()
        self.get_dtype = P.DType()
        self.reshape = P.Reshape()
        self.get_shape = P.Shape()

    def construct(self, input_tensor):
        """layer norm"""
        shape = self.get_shape(input_tensor)
        batch_size = shape[0]
        max_len = shape[1]
        embed_dim = shape[2]

        output = self.reshape(input_tensor, (-1, embed_dim))
        output = self.cast(output, mstype.float32)
        output = self.layer_norm(output)
        output = self.cast(output, self.get_dtype(input_tensor))
        if not self.return_2d:
            output = self.reshape(output, (batch_size, max_len, embed_dim))
        return output

#### 2.2.4 ResidualConnection
ResidualConnection类实现一个残差结构，能有效解决梯度消失的问题。

In [6]:
class ResidualConnection(nn.Cell):

    def __init__(self, dropout_prob=0.1):
        super(ResidualConnection, self).__init__()
        self.add = P.Add()
        self.dropout = nn.Dropout(1.0 - dropout_prob)

    def construct(self, hidden_tensor, residual):
        output = self.dropout(hidden_tensor)
        output = self.add(output, residual)
        return output

### 2.3 FeedForward

网络结构图中的Feed Forward -> Add & Norm 在FeedForwardNet类中实现。<br>
先进行Layer Normalization 然后经过两个全连接层，最后加上残差结构输出。

<div align=center>
    <img src='https://i.imgur.com/5KQFzmm.png' width='550px'>
</div>

In [7]:
class FeedForwardNet(nn.Cell):
    def __init__(self,
                 in_channels,
                 hidden_size,
                 out_channels,
                 hidden_act="relu",
                 initializer_range=0.02,
                 hidden_dropout_prob=0.1,
                 dropout=None,
                 compute_type=mstype.float32):
        super(FeedForwardNet, self).__init__()

        self.fc1 = nn.Dense(in_channels,
                            hidden_size,
                            activation=hidden_act,
                            weight_init=TruncatedNormal(initializer_range)).to_float(compute_type)
        self.fc2 = nn.Dense(hidden_size,
                            out_channels,
                            weight_init=TruncatedNormal(initializer_range)).to_float(compute_type)

        self.layer_norm = LayerNorm(in_channels=in_channels,
                                    return_2d=True)
        self.residual = ResidualConnection(
            dropout_prob=hidden_dropout_prob if dropout is None else dropout
        )
        self.get_shape = P.Shape()
        self.reshape = P.Reshape()
        self.dropout = nn.Dropout(keep_prob=1.0 - hidden_dropout_prob)

    def construct(self, input_tensor):
        shape = self.get_shape(input_tensor)
        batch_size = shape[0]
        max_len = shape[1]
        embed_dim = shape[2]
        # 核心代码如下
        output = self.layer_norm(input_tensor)
        output = self.fc1(output)
        output = self.dropout(output)
        output = self.fc2(output)  # (-1, D)
        output = self.residual(self.reshape(output, (batch_size, max_len, embed_dim)),
                               input_tensor)  # (N, T, D)
        return output


### 2.4 编码器与解码器

<div align=center><img src='https://i.imgur.com/NNiUhIt.png' width='350px'/></div>

可以看到编码器与解码器是由几个模块构成的，分别是Multi-Head Attention、Feed Forward、Masked Multi-Head Attention。

#### 2.4.1 编码器

编码器包含 6 个完全相同的神经网路层，每一个网路层包含了 2 个子层，第一个子层是 「multi-head self-attention」 层，第二个子层是 「positionwise fully connected feed-forward」 层。在每个层后还有一个正规化与残差运算。

用数学式来看，每一个网路层可以表示为：<br>
输入: $x$ <br>
第一个子层输出: $y=LayerNorm(x+MHA(x))$ <br>
第二个子层输出: $O=LayerNorm(y+PFF(y))$ <br>

```
编码器单层结构如下：
    -> pre_LayerNorm
    -> Multi-head Self-Attention
    -> Dropout & Add
    -> pre_LayerNorm
    -> Fc1
    -> Activation Function
    -> Dropout
    -> Fc2
    -> Dropout & Add
```

EncoderCell类就是按顺序调用SelfAttention类与FeedForward类。

In [9]:
class EncoderCell(nn.Cell):
    """Single Encoder layer."""
    def __init__(self,
                 attn_embed_dim=768,
                 num_attn_heads=12,
                 intermediate_size=3072,
                 attention_dropout_prob=0.02,
                 initializer_range=0.02,
                 hidden_dropout_prob=0.1,
                 hidden_act="relu",
                 compute_type=mstype.float32):
        super(EncoderCell, self).__init__()
        self.attention = SelfAttention(
            attn_embed_dim=attn_embed_dim,
            num_attn_heads=num_attn_heads,
            attn_dropout_prob=attention_dropout_prob,
            initializer_range=initializer_range,
            dropout_prob=hidden_dropout_prob,
            compute_type=compute_type)
        self.feed_forward_net = FeedForwardNet(
            in_channels=attn_embed_dim,
            hidden_size=intermediate_size,
            out_channels=attn_embed_dim,
            hidden_act=hidden_act,
            initializer_range=initializer_range,
            hidden_dropout_prob=hidden_dropout_prob,
            dropout=hidden_dropout_prob,
            compute_type=compute_type)

    def construct(self, queries, attention_mask):
        attention_output = self.attention(queries, queries, queries,
                                          attention_mask)  # (N, T, D)
        output = self.feed_forward_net(attention_output)  # (N, T, D)
        return output

TransformerEncoder类的功能是重复构造EncoderCell类，然后顺序连接起来，对应网络框架图中的Encoder部分的Nx结构，实际上是重复了6次。

<div align=center><img src='https://i.imgur.com/DMbU83g.png' width='400px'/></div>



In [10]:
class TransformerEncoder(nn.Cell):
    def __init__(self,
                 attn_embed_dim,
                 encoder_layers,
                 num_attn_heads=12,
                 intermediate_size=3072,
                 attention_dropout_prob=0.1,
                 initializer_range=0.02,
                 hidden_dropout_prob=0.1,
                 hidden_act="relu",
                 compute_type=mstype.float32):
        super(TransformerEncoder, self).__init__()
        self.num_layers = encoder_layers

        layers = []
        for _ in range(encoder_layers):
            layer = EncoderCell(
                attn_embed_dim=attn_embed_dim,
                num_attn_heads=num_attn_heads,
                intermediate_size=intermediate_size,
                attention_dropout_prob=attention_dropout_prob,
                initializer_range=initializer_range,
                hidden_dropout_prob=hidden_dropout_prob,
                hidden_act=hidden_act,
                compute_type=compute_type
            )
            layers.append(layer)
        # num_attention_heads的值为8，通过nn.CellList顺序连接起来。
        self.layers = nn.CellList(layers)
        self.layer_norm = LayerNorm(in_channels=attn_embed_dim)

    def construct(self, input_tensor, attention_mask):
        prev_output = input_tensor
        for layer_module in self.layers:
            prev_output = layer_module(prev_output,
                                       attention_mask)  # (N, T, D)
        prev_output = self.layer_norm(prev_output)  # (N, T, D)
        return prev_output

#### 2.4.2 解码器

解码器也同样包含了 6 个相同的神经网路层，每一个网路层含有 3 个子层，由下到上分别是：

>Masked Multi-Head Self-Attention <br>
>Multi-Head Attention <br>
>Positionwise Fully Connected Feed-Forward

每一个子层也实作残差连接及正规化处理。

Transformer 解码器的架构和编码器雷同，但有两个主要的差异：
- 解码器每一层的第一个子层必须执行 Masked 动作，使用 Masked Multi-Head Self-Attention 是为了确保在计算第 i 个输出时，仅用到 i 之前的输出值。
- 解码器每一层都多了一个 Multi-Head Attention 子层，该层是以“第一个子层的输出”和“编码器的输出”执行 Attention 计算。

CreateAttentionMaskFromInputMask类实现Mask Self Attention中的Mask功能。

In [ ]:
class CreateAttentionMaskFromInputMask(nn.Cell):
    
    def __init__(self, config):
        super(CreateAttentionMaskFromInputMask, self).__init__()
        self.input_mask = None

        self.cast = P.Cast()
        self.shape = P.Shape()
        self.reshape = P.Reshape()
        self.batch_matmul = P.BatchMatMul()
        self.multiply = P.Mul()
        self.shape = P.Shape()
        # mask future positions
        ones = np.ones(shape=(config.batch_size, config.seq_length, config.seq_length))
        self.lower_triangle_mask = Tensor(np.tril(ones), dtype=mstype.float32)

    def construct(self, input_mask, mask_future=False):
        input_shape = self.shape(input_mask)
        # Add this for infer as the seq_length will increase.
        shape_right = (input_shape[0], 1, input_shape[1])
        shape_left = input_shape + (1,)

        input_mask = self.cast(input_mask, mstype.float32)
        mask_left = self.reshape(input_mask, shape_left)
        mask_right = self.reshape(input_mask, shape_right)

        attention_mask = self.batch_matmul(mask_left, mask_right)

        if mask_future:
            attention_mask = self.multiply(attention_mask, self.lower_triangle_mask)

        return attention_mask

DecoderCell类中依序调用Self Attention类及FeedForward类。<br>
其中第二个子层 Multi-Head Attention，它并不是一个 Self-Attention，它的输入 V 及 K 是来自编码器的输出值，而 Q 才是来自第一个子层。因此这一个子层在论文中，有时称它为 encoder-decoder attention (sub)layer，论文中说明这种设计可以让每一个解码器子层的输入值去关注编码器的输出序列的每个位置。
> In "encoder-decoder attention" layers, the queries come from the previous decoder layer, and the memory keys and values come from the output of the encoder. This allows every position in the decoder to attend over all positions in the input sequence. This mimics the typical encoder-decoder attention mechanisms in sequence-to-sequence models.

In [ ]:
class DecoderCell(nn.Cell):

    def __init__(self,
                 attn_embed_dim=768,
                 num_attn_heads=12,
                 intermediate_size=3072,
                 attn_dropout_prob=0.02,
                 initializer_range=0.02,
                 dropout_prob=0.1,
                 hidden_act="relu",
                 compute_type=mstype.float32):
        super(DecoderCell, self).__init__()
        self.masked_attn = SelfAttention(
            attn_embed_dim=attn_embed_dim,
            num_attn_heads=num_attn_heads,
            attn_dropout_prob=attn_dropout_prob,
            initializer_range=initializer_range,
            dropout_prob=dropout_prob,
            compute_type=compute_type)
        self.enc_dec_attn = SelfAttention(
            attn_embed_dim=attn_embed_dim,
            num_attn_heads=num_attn_heads,
            attn_dropout_prob=attn_dropout_prob,
            initializer_range=initializer_range,
            dropout_prob=dropout_prob,
            compute_type=compute_type)
        self.feed_forward_net = FeedForwardNet(
            in_channels=attn_embed_dim,
            hidden_size=intermediate_size,
            out_channels=attn_embed_dim,
            hidden_act=hidden_act,
            initializer_range=initializer_range,
            hidden_dropout_prob=dropout_prob,
            compute_type=compute_type)

    def construct(self, queries, attention_mask, encoder_out, enc_attention_mask):
        attention_output = self.masked_attn(
            queries, queries, queries,
            attention_mask
        )
        attention_output = self.enc_dec_attn(
            attention_output,  # (N, T', D)
            encoder_out, encoder_out,  # (N, T, D)
            enc_attention_mask  # (N, T, T)
        )
        output = self.feed_forward_net(attention_output)
        return output

TransformerDecoder类的功能是重复构造DecoderCell类，然后顺序连接起来，对应网络框架图中的Decoder部分的Nx结构，实际上是重复了6次。


<div align=center><img src='https://i.imgur.com/bdu2GMc.png' width='400px'/></div>




In [ ]:
class TransformerDecoder(nn.Cell):
    def __init__(self,
                 attn_embed_dim,
                 decoder_layers,
                 num_attn_heads,
                 intermediate_size,
                 attn_dropout_prob=0.1,
                 initializer_range=0.02,
                 dropout_prob=0.1,
                 hidden_act="relu",
                 compute_type=mstype.float32):
        super(TransformerDecoder, self).__init__()
        self.num_layers = decoder_layers
        self.attn_embed_dim = attn_embed_dim

        self.layer0 = DecoderCell(
            attn_embed_dim=attn_embed_dim,
            num_attn_heads=num_attn_heads,
            intermediate_size=intermediate_size,
            attn_dropout_prob=attn_dropout_prob,
            initializer_range=initializer_range,
            dropout_prob=dropout_prob,
            hidden_act=hidden_act,
            compute_type=compute_type
        )
        self.layer1 = DecoderCell(
            attn_embed_dim=attn_embed_dim,
            num_attn_heads=num_attn_heads,
            intermediate_size=intermediate_size,
            attn_dropout_prob=attn_dropout_prob,
            initializer_range=initializer_range,
            dropout_prob=dropout_prob,
            hidden_act=hidden_act,
            compute_type=compute_type
        )
        self.layer2 = DecoderCell(
            attn_embed_dim=attn_embed_dim,
            num_attn_heads=num_attn_heads,
            intermediate_size=intermediate_size,
            attn_dropout_prob=attn_dropout_prob,
            initializer_range=initializer_range,
            dropout_prob=dropout_prob,
            hidden_act=hidden_act,
            compute_type=compute_type
        )
        self.layer3 = DecoderCell(
            attn_embed_dim=attn_embed_dim,
            num_attn_heads=num_attn_heads,
            intermediate_size=intermediate_size,
            attn_dropout_prob=attn_dropout_prob,
            initializer_range=initializer_range,
            dropout_prob=dropout_prob,
            hidden_act=hidden_act,
            compute_type=compute_type
        )
        self.layer4 = DecoderCell(
            attn_embed_dim=attn_embed_dim,
            num_attn_heads=num_attn_heads,
            intermediate_size=intermediate_size,
            attn_dropout_prob=attn_dropout_prob,
            initializer_range=initializer_range,
            dropout_prob=dropout_prob,
            hidden_act=hidden_act,
            compute_type=compute_type
        )
        self.layer5 = DecoderCell(
            attn_embed_dim=attn_embed_dim,
            num_attn_heads=num_attn_heads,
            intermediate_size=intermediate_size,
            attn_dropout_prob=attn_dropout_prob,
            initializer_range=initializer_range,
            dropout_prob=dropout_prob,
            hidden_act=hidden_act,
            compute_type=compute_type
        )

        self.layer_preprocess = LayerNorm(in_channels=attn_embed_dim,
                                          return_2d=False)

    def construct(self, input_tensor, attention_mask, encoder_out, enc_attention_mask):
        prev_output = input_tensor
        prev_output = self.layer0(prev_output, attention_mask, encoder_out, enc_attention_mask)
        prev_output = self.layer1(prev_output, attention_mask, encoder_out, enc_attention_mask)
        prev_output = self.layer2(prev_output, attention_mask, encoder_out, enc_attention_mask)
        prev_output = self.layer3(prev_output, attention_mask, encoder_out, enc_attention_mask)
        prev_output = self.layer4(prev_output, attention_mask, encoder_out, enc_attention_mask)
        prev_output = self.layer5(prev_output, attention_mask, encoder_out, enc_attention_mask)

        # Add layer norm, and full connection layer.
        prev_output = self.layer_preprocess(prev_output)
        return prev_output

### 2.5 SoftMax
解码器的输出是一个浮点数向量，经过一个解码器中最后的Linear 层，输出为一个 logits 向量，此一向量的长度即是字汇数量，每一个位置代表一个字。假设我们的输出为一 10000 字的英文字汇集合，那么 logit 向量的长度即为 10000。

再经过 Softmax 处理，则输出为一机率分布向量，代表每一个对应文字的机率。通常最高机率的那一个字，即被选为此次的输出单字。

PredLogProbs类实现SoftMax功能，对应网络框架图中Decoder部分最后输出之前的SoftMax部分。

<div align=center>
<img src='https://i.imgur.com/i0Vbk7r.png' width=150/>
</div>



In [ ]:
class PredLogProbs(nn.Cell):
    def __init__(self,
                 batch_size,
                 seq_length,
                 width,
                 compute_type=mstype.float32,
                 dtype=mstype.float32):
        super(PredLogProbs, self).__init__()
        self.batch_size = batch_size
        self.seq_length = seq_length
        self.width = width
        self.compute_type = compute_type
        self.dtype = dtype

        self.reshape = P.Reshape()
        self.matmul = P.MatMul(transpose_b=True)
        self.log_softmax = nn.LogSoftmax(axis=-1)
        self.shape_flat_sequence_tensor = (self.batch_size * self.seq_length, self.width)
        self.cast = P.Cast()

    def construct(self, input_tensor, output_weights):
        input_tensor = self.reshape(input_tensor, self.shape_flat_sequence_tensor)
        input_tensor = self.cast(input_tensor, self.compute_type)
        output_weights = self.cast(output_weights, self.compute_type)

        logits = self.matmul(input_tensor, output_weights)
        logits = self.cast(logits, self.dtype)

        log_probs = self.log_softmax(logits)
        return log_probs

### 2.6 Transformer
Transformer类连接编码器和解码器，并分别在其输入前加上 Embedding & Positional Encoding，并在解码器的输出处加上 Linear 层及 Softmax 层，实现从输入到输出的端到端流程。

In [ ]:
class Transformer(nn.Cell):

    def __init__(self):
        super(Transformer, self).__init__()

        self.batch_size = config.batch_size
        self.max_positions = config.seq_length
        self.attn_embed_dim = config.hidden_size
        self.num_layers = config.num_hidden_layers
        self.word_embed_dim = config.hidden_size

        self.last_idx = self.num_layers - 1

        self.embedding_lookup = EmbeddingLookup(
            vocab_size=config.vocab_size,
            embed_dim=self.word_embed_dim,
            use_one_hot_embeddings=False)

        self.positional_embedding = PositionalEmbedding(
            embedding_size=self.word_embed_dim,
            max_position_embeddings=config.max_position_embeddings)

        self.encoder = TransformerEncoder(
            attn_embed_dim=self.attn_embed_dim,
            encoder_layers=self.num_layers,
            num_attn_heads=config.num_attention_heads,
            intermediate_size=config.intermediate_size,
            attention_dropout_prob=config.attention_dropout_prob,
            initializer_range=config.initializer_range,
            hidden_dropout_prob=config.hidden_dropout_prob,
            hidden_act=config.hidden_act,
            compute_type=config.compute_type)

        self.decoder = TransformerDecoder(
            attn_embed_dim=self.attn_embed_dim,
            decoder_layers=self.num_layers,
            num_attn_heads=config.num_attention_heads,
            intermediate_size=config.intermediate_size,
            attn_dropout_prob=config.attention_dropout_prob,
            initializer_range=config.initializer_range,
            dropout_prob=config.hidden_dropout_prob,
            hidden_act=config.hidden_act,
            compute_type=config.compute_type)

        self.cast = P.Cast()
        self.dtype = config.dtype
        self.cast_compute_type = SaturateCast(dst_type=config.compute_type)
        self.slice = P.StridedSlice()
        self.dropout = nn.Dropout(keep_prob=1.0 - config.hidden_dropout_prob)

        self._create_attention_mask_from_input_mask = CreateAttentionMaskFromInputMask(config)

        self.scale = Tensor([math.sqrt(float(self.word_embed_dim))],
                            dtype=mstype.float32)
        self.multiply = P.Mul()

    def construct(self, source_ids, source_mask, target_ids, target_mask):
        # Process source sentences.
        src_embeddings, embedding_tables = self.embedding_lookup(source_ids)
        src_embeddings = self.multiply(src_embeddings, self.scale)
        if self.use_positional_embedding:
            src_embeddings = self.positional_embedding(src_embeddings)
        src_embeddings = self.dropout(src_embeddings)

        # Attention mask with shape (N, T, T).
        enc_attention_mask = self._create_attention_mask_from_input_mask(source_mask)
        # Transformer encoder.
        encoder_output = self.encoder(
            self.cast_compute_type(src_embeddings),  # (N, T, D).
            self.cast_compute_type(enc_attention_mask)  # (N, T, T).
        )

        # Process target sentences.
        tgt_embeddings, _ = self.embedding_lookup(target_ids)
        tgt_embeddings = self.multiply(tgt_embeddings, self.scale)
        if self.use_positional_embedding:
            tgt_embeddings = self.positional_embedding(tgt_embeddings)
        tgt_embeddings = self.dropout(tgt_embeddings)

        # Attention mask with shape (N, T', T').
        tgt_attention_mask = self._create_attention_mask_from_input_mask(
            target_mask, True
        )
        # Transformer decoder.
        decoder_output = self.decoder(
            self.cast_compute_type(tgt_embeddings),  # (N, T', D)
            self.cast_compute_type(tgt_attention_mask),  # (N, T', T')
            encoder_output,  # (N, T, D)
            enc_attention_mask  # (N, T, T)
        )

        return encoder_output, decoder_output, embedding_tables

注：本节中的模型图，除了论文中的图片，其他皆源自 Jay Alammar 的[《The Illustrated Transformer》](https://jalammar.github.io/illustrated-transformer/)

## 3 案例实现

### 3.1 环境建置
本案例的开发环境为Modelarts上的notebook。<br>
版本：mindspore 1.8.1、python 3.7<br>
Ascend 环境：1\*Ascend 910 | CPU: 24vCPUs 96GB<br>

In [11]:
import mindspore
print(mindspore.run_check())

MindSpore version:  1.8.1
The result of multiplication calculation is correct, MindSpore has been installed successfully!
None


进入开发环境后还需安装一些所需套件，执行以下命令即可：

In [ ]:
!pip install -r requirements.txt

In [ ]:
import nltk
nltk.download('punkt')

若是nltk的punkt套件无法下载的话请至[该连接](https://github.com/nltk/nltk_data/tree/gh-pages/packages/tokenizers)下载punkt.zip，解压后将punkt资料夹放入/root/nltk_data/tokenizers/路径下。<br>
ModelArts环境下放置在/home/ma-user/nltk_data/tokenizers/路径下，步骤为：将下载好的punkt.zip上传至work目录中，然後在终端输入如下命令。
```
[ma-user work]$ cd ..
[ma-user ~]$ mkdir nltk_data
[ma-user ~]$ mkdir nltk_data/tokenizers
[ma-user ~]$mv work/punkt.zip nltk_data/tokenizers
[ma-user ~]$cd nltk_data/tokenizers
[ma-user tokenizers]$unzip punkt.zip
```

#### 3.1.1 代码结构

MASS脚本及代码结构如下：

```text
├── mass
  ├── config
  │   ├──config.py                           // 参数配置
  ├── src
      │   ├──model_utils
      │   ├──config.py                       // 参数配置
      │   ├──device_adapter.py               // 设备配置
      │   ├──local_adapter.py                // 本地设备配置
      │   ├──moxing_adapter.py               // modelarts设备配置
  ├──src
  │   ├──dataset
  │      ├──bi_data_loader.py                // 数据集加载器，用于微调或推理
  │      ├──mono_data_loader.py              // 预训练数据集加载器
  │   ├──language_model
  │      ├──noise_channel_language_model.p   // 数据集生成噪声通道语言模型
  │      ├──mass_language_model.py           // 基于MASS论文的MASS语言模型
  │      ├──loose_masked_language_model.py   // 基于MASS发布代码的MASS语言模型
  │      ├──masked_language_model.py         // 基于MASS论文的MASS语言模型
  │   ├──transformer
  │      ├──create_attn_mask.py              // 生成屏蔽矩阵，除去填充部分
  │      ├──transformer.py                   // Transformer模型架构
  │      ├──encoder.py                       // Transformer编码器组件
  │      ├──decoder.py                       // Transformer解码器组件
  │      ├──self_attention.py                // 自注意块组件
  │      ├──multi_head_attention.py          // 多头自注意组件
  │      ├──embedding.py                     // 嵌入组件
  │      ├──positional_embedding.py          // 位置嵌入组件
  │      ├──feed_forward_network.py          // 前馈网络
  │      ├──residual_conn.py                 // 残留块
  │      ├──beam_search.py                   // 推理所用的波束搜索解码器
  │      ├──transformer_for_infer.py         // 使用Transformer进行推理
  │      ├──transformer_for_train.py         // 使用Transformer进行训练
  │   ├──utils
  │      ├──byte_pair_encoding.py            // 使用subword-nmt应用字节对编码（BPE）
  │      ├──dictionary.py                    // 字典
  │      ├──loss_moniter.py                  // 训练步骤中损失监控回调
  │      ├──lr_scheduler.py                  // 学习速率调度器
  │      ├──ppl_score.py                     // 基于N-gram的困惑度评分
  │      ├──rouge_score.py                   // 计算ROUGE得分
  │      ├──load_weights.py                  // 从检查点或者NPZ文件加载权重
  │      ├──initializer.py                   // 参数初始化器
  ├── vocab
  │   ├──all.bpe.codes                       // 字节对编码表
  │   ├──all_en.dict.bin                     // 已学习到的词汇表
  ├── scripts
  │   ├──run_ascend.sh                       // Ascend处理器上训练&评估模型脚本
  │   ├──run_gpu.sh                          // GPU处理器上训练&评估模型脚本
  │   ├──learn_subword.sh                    // 学习字节对编码
  │   ├──stop_training.sh                    // 停止训练
  ├── requirements.txt                       // 第三方包需求
  ├── train.py                               // 训练API入口
  ├── eval.py                                // 推理API入口
  ├── default_config.yaml                    // 参数配置
  ├── tokenize_corpus.py                     // 语料标记化
  ├── apply_bpe_encoding.py                  // 应用BPE进行编码
  ├── weights_average.py                     // 将各模型检查点平均转换到NPZ格式
  ├── news_crawl.py                          // 创建预训练所用的News Crawl数据集
  ├── gigaword.py                            // 创建Gigaword语料库
```


### 3.2 准备数据集
案例实现中预训练所使用的数据即News Crawl的英语单语数据数据集，因為考量到訓練模型所需時間，只使用2012年的數據(News Crawl: articles from 2012)，下载好的数据集为一纯文字文件，接下来需要对该数据进行预处理，预处理包括对数据进行分词、利用subword-nmt工具生成字节对编码(BPE)、对分词后的语料应用该BPE编码進行切分并构建词彙表幾個步驟。

而微调模型用于文本摘要任务所使用的数据集为Gigaword，该数据集已经有分割为训练、测试、验证集，有原文本(src)和目标摘要(tgt)两个文件，本案例只会使用训练集与测试集，也需要對這兩個文件進行同樣的預處理。

数据集文件路径结构如下：

```text
.dataset/<br>
└── news_crawl<br>
    └── news2012.txt(1.8GB)
└── ggw_data<br>
    ├── test.src.txt(335KB)
    ├── test.tgt.txt(104KB)
    ├── train.src.txt(695.2MB)
    └── train.tgt.txt(198.5MB)
```
下载连接：<br>
News Crawl数据集（WMT，2019年）：https://www.statmt.org/wmt16/translation-task.html<br>
Gigaword语料库（Graff等人，2003年）： https://drive.google.com/open?id=0B6N7tANPyVeBNmlSX19Ld2xDU1E



#### 3.2.1 对原始数据进行分词

文本是由段落（Paragraph）构成的，段落是由句子（Sentence）构成的，句子是由单词构成的。分词是文本分析的第一步，它把文本段落分解为较小的实体（如单词或句子），每一个实体叫做一个Token。NLTK套件能够实现句子切分和单词切分两种功能。

因为原始数据是一行一行句子的格式，所以只要调用nltk套件中的word_tokenize()函数文本进行分词，此函数的作用是基于空格/标点等对文本进行分词，返回分词后的列表，最后只保留长度大于175的单词列表，并将这些单词列表写入目标文件中。

```
sen = "Dave Aneckstein, Simmons Research, an Experian Company"
tokens = word_tokenize(sen)
tokens = ['Dave', 'Aneckstein', ',', 'Simmons', 'Research', ',', 'an', 'Experian', 'Company']
```





In [18]:
import os
from nltk.tokenize import word_tokenize

data_list = ['news_crawl', 'ggw_data']
src_folder = "./dataset/"
out_folder = "./tokenized_corpus/"

if not os.path.isdir(out_folder):
    os.mkdir(out_folder)
if not os.path.isdir(out_folder+'news_crawl/'):
    os.mkdir(out_folder+'news_crawl/')
if not os.path.isdir(out_folder+'ggw_data/'):
    os.mkdir(out_folder+'ggw_data/')

def create_tokenized_sentences(file_path, tokenized_file):
    tokenized_sen = []
    print(f" | Processing {file_path}.")
    with open(file_path, "r") as file:
        for sen in file:
            tokens = word_tokenize(sen)
            tokens = [t for t in tokens if t != " "]
            if len(tokens) > 175:
                continue
            tokenized_sen.append(" ".join(tokens) + "\n")

    with open(tokenized_file, "w") as file:
        file.writelines(tokenized_sen)
    print(f" | Wrote to {tokenized_file}.")

for item in data_list:
    folder_path = os.path.join(src_folder, item)
    output_path = os.path.join(out_folder, item)
    for file in os.listdir(folder_path):
        if not file.endswith(".txt"):
            continue
        file_path = os.path.join(folder_path, file)
        tokenized_file = os.path.join(output_path, file.replace(".txt", "_tokenized.txt"))
        create_tokenized_sentences(file_path, tokenized_file)

 | Processing ./dataset/news_crawl/news2007.txt.
 | Wrote to ./tokenized_corpus/news_crawl/news2007_tokenized.txt.
 | Processing ./dataset/ggw_data/test.src.txt.
 | Wrote to ./tokenized_corpus/ggw_data/test.src_tokenized.txt.
 | Processing ./dataset/ggw_data/train.src.txt.
 | Wrote to ./tokenized_corpus/ggw_data/train.src_tokenized.txt.
 | Processing ./dataset/ggw_data/train.tgt.txt.
 | Wrote to ./tokenized_corpus/ggw_data/train.tgt_tokenized.txt.
 | Processing ./dataset/ggw_data/test.tgt.txt.
 | Wrote to ./tokenized_corpus/ggw_data/test.tgt_tokenized.txt.


#### 3.2.2 生成字节对编码
虽然我们已经将数据进行了简单的分词，但是实际上这样的分词是不够细緻的，若要使用一个包含所有单词的字典，将需要很大的空间和计算量，而且过大的token列表也会影响模型的预测准确度。

字节对编码（BPE, Byte Pair Encoder）是一种简单的数据压缩算法，它将字符串中出现频率最高的相邻字符替换成一个不存在的新字符，反复进行该操作直到满足某些预设条件为止（字符表大小、迭代次数）。后续使用时还需要一个词表来重建原始数据。

BPE的处理过程可以理解为一个单词的再拆分过程。如"loved","loving","loves"这三个单词，其本身的语义都是”爱”的意思。BPE通过训练，能够把上面的3个单词拆分成”lov”,”ed”,”ing”,”es”几部分，这样可以把词的本身的意思和时态分开，有效的减少了词表大小，且罕见的词会被分解为两个或多个subword tokens，能比较好的处理OOV(out of vocabulary)问题。

我们可以利用subword-nmt工具学习文本的字节对编码(learn-bpe)：

先设定选择最高频字节对的次数num_operations，本案例设定为46000。此过程会统计每一个相邻字节对的出现频率，并保存为code_file。
```
subword-nmt learn-bpe -i {input_src_file} -s {num_operations} -o {codes_file}
```


In [1]:
!cat ./dataset/ggw_data/*.txt ./dataset/news_crawl/*.txt | subword-nmt learn-bpe -s 46000 -o ./dataset/all.bpe.codes

100%|#####################################| 46000/46000 [16:34<00:00, 46.26it/s]


#### 3.2.3 应用字节对编码并构建词彙表
使用上一节获取的BPE编码对分词后的语料进行字节对编码处理(apply-bpe)，生成subword词表。此过程会将input_tokenized_file中的单词拆分为字符序列并在末尾添加后缀“<\/w>”，而后按照code_file将出现频率最高的字节对合并成新的subword，重複合併直到达到设定的subword词表大小或下一个最高频的字节对出现频率为1，最后将结果保存为out_file，文件名为input_tokenized_file_bpe.txt。
```
subword-nmt apply-bpe -c {code_file} < {input_tokenized_file} > {out_file}
可以通过 --vocabulary-threshold {threshold} 選項过滤词频低于阈值的单词来缩小词汇量。
```
构建词彙表(get-vocab)的具体作法为：得到subword词表后，对该词表按照子词长度由大到小排序。编码时，对于每个单词，遍历词表寻找是否有token是当前单词的子字符串，如果有，则该token为表示当前单词的tokens之一。从最长的token迭代到最短的token，尝试将每个单词中的子字符串替换为已存在的token。若每个单词都替换完后仍然有子字符串没被替换，则将剩余的子词替换为特殊token，如\<unk>。
```
subword-nmt get-vocab -i {output_path} -o {dict_path}
# output_path為apply-bpe的輸出路徑
# dict_path為輸出的.dict檔案路徑
```

根据News Crawl 2012数据集的14869673个句子，学习到的词汇量为39284个单词。

In [2]:
import os
import subprocess
from src.utils import Dictionary

source_folder = os.path.abspath("./tokenized_corpus/news_crawl/")
output_folder = os.path.abspath("./tokenized_corpus/news_crawl/bpe/")
codes = os.path.abspath("./dataset/all.bpe.codes")
vocab_path = "./vocab/all_en.dict.bin"

if not os.path.isdir(output_folder):
    os.mkdir(output_folder)
if not os.path.isdir('./vocab/'):
    os.mkdir('./vocab/')


ENCODER = "subword-nmt apply-bpe -c"
LEARN_DICT = "subword-nmt get-vocab -i"
def bpe_encode(codes_path, src_path, output_path, dict_path):
    # Encoding.
    print(" | Applying BPE encoding.")
    commands = ENCODER.split() + [codes_path] + ["-i"] + [src_path] + ["-o"] + [output_path]
    subprocess.call(commands)
    print(" | Fetching vocabulary from single file.")
    # Learn vocab.
    commands = LEARN_DICT.split() + [output_path] + ["-o"] + [dict_path]
    subprocess.call(commands)

available_dict = []
for file in os.listdir(source_folder):
    if file.endswith(".txt"):
        output_path = os.path.join(output_folder, file.replace(".txt", "_bpe.txt"))
        dict_path = os.path.join(output_folder, file.replace(".txt", ".dict"))
        available_dict.append(dict_path)
        bpe_encode(codes, os.path.join(source_folder, file), output_path, dict_path)
        
# 加载bpe_encode處理過的文本词汇表，行格式为[word, freq]。
vocab = Dictionary.load_from_text(available_dict)
vocab.persistence(vocab_path) #将词汇表对象保存为二进制文件。
print(f" | Vocabulary Size: {len(vocab)}")

 | Applying BPE encoding.
 | Fetching vocabulary from single file.
 | Vocabulary Size: 38086


### 3.3 生成数据集

因为生成的数据集格式是tfrecord, 这边需要将kernal修改为tensorflow-1.15，再安装mindspore。
并使用以下命令处理下某些套件与tensorflow的冲突。

In [1]:
!pip install mindspore-ascend==1.8.1 rouge
!pip uninstall -y urllib3 chardet
!pip install --upgrade requests
# !pip uninstall -y mindspore

Looking in indexes: http://192.168.0.122:8888/repository/pypi/simple
Found existing installation: urllib3 1.26.12
Uninstalling urllib3-1.26.12:
  Successfully uninstalled urllib3-1.26.12
Found existing installation: chardet 4.0.0
Uninstalling chardet-4.0.0:
  Successfully uninstalled chardet-4.0.0
Looking in indexes: http://192.168.0.122:8888/repository/pypi/simple


In [ ]:
"""Create News Crawl Pre-Training Dataset."""
import os
from src.dataset import MonoLingualDataLoader
from src.language_model import LooseMaskedLanguageModel
from src.utils import Dictionary

input_folder_path = './dataset/news_crawl/' # Raw corpus folder
output_folder_path = './train_data/news_crawl/' # Dataset output path

if not os.path.isdir(output_folder_path):
    os.mkdir('./train_data')
    os.mkdir('./train_data/news_crawl/')

vocab_path = './vocab/all_en.dict.bin' # Existed vocab path
vocab = Dictionary.load_from_persisted_dict(vocab_path)

def create_pre_train(text_file, output_folder, vocab, max_sen_len):
    
    loader = MonoLingualDataLoader(
        src_filepath=text_file,
        lang="en", dictionary=vocab,
        language_model=LooseMaskedLanguageModel(mask_ratio=0.5, mask_all_prob=None),
        max_sen_len=max_sen_len, min_sen_len=10
    )

    src_file_name = os.path.basename(text_file)

    file_name = os.path.join(
        output_folder_path,
        src_file_name.replace('.txt', f'_len_{max_sen_len}.tfrecord')
    )
    
    loader.write_to_tfrecord(path=file_name)

for file in os.listdir(input_folder_path):
    if file.endswith(".txt"):
        create_pre_train(os.path.join(input_folder_path, file),output_folder_path, vocab, 64)

print(f" | Generate Dataset for Pre-training is done.")
print(f" | Vocabulary size: {vocab.size}.")

 | Processing corpus ./dataset/news_crawl/news2007.txt.
 | Shortest len = 1.
 | Longest  len = 3269.
 | Total    sen = 3163480.


In [ ]:
"""Generate Gigaword dataset."""
import os
from src.dataset import BiLingualDataLoader
from src.language_model import NoiseChannelLanguageModel
from src.utils import Dictionary

input_folder_path = './dataset/ggw_data/'
output_folder_path = './train_data/ggw_data/'

if not os.path.isdir(output_folder_path):
    os.mkdir('./train_data/ggw_data/')

vocab_path = './vocab/all_en.dict.bin'
vocab = Dictionary.load_from_persisted_dict(vocab_path)

train = BiLingualDataLoader(
    src_filepath=os.path.join(input_folder_path,"train.src.txt"),
    tgt_filepath=os.path.join(input_folder_path,"train.tgt.txt"),
    src_dict=vocab, tgt_dict=vocab,
    src_lang="en", tgt_lang="en",
    language_model=NoiseChannelLanguageModel(add_noise_prob=0.),
    max_sen_len=32
)

train.write_to_tfrecord(
    path=os.path.join(output_folder_path, "gigaword_train_dataset.tfrecord")
)

test = BiLingualDataLoader(
    src_filepath=os.path.join(input_folder_path,"test.src.txt"),
    tgt_filepath=os.path.join(input_folder_path,"test.tgt.txt"),
    src_dict=vocab, tgt_dict=vocab,
    src_lang="en", tgt_lang="en",
    language_model=NoiseChannelLanguageModel(add_noise_prob=0),
    max_sen_len=32
)

test.write_to_tfrecord(
    path=os.path.join(output_folder_path, "gigaword_test_dataset.tfrecord")
)

print(f" | Generate Dataset for fine-tuneing is done.")
print(f" | Vocabulary size: {vocab.size}.")

生成数据集的步骤完成后再切换回mindspore的kernal，再安装一次所需套件：

In [2]:
!pip install -r requirements.txt

Looking in indexes: http://192.168.0.122:8888/repository/pypi/simple


### 3.4 预训练

预训练中，采用Adam优化器和损失放大来得到预训练后的模型。






In [ ]:
"""定义训练与推理的参数"""
import mindspore.common.dtype as mstype
class config():
    enable_modelarts = True #Whether training on modelarts, default = False
    device_target = "Ascend"
    output_path = "./output/"
    save_checkpoint_path = "./output/checkpoint/"
    checkpoint_file_path = ""
    # Training options
    epochs = 20
    batch_size = 192
    dtype = mstype.float32 #only support float16 and float32
    compute_type = mstype.float16 #only support float16 and float32
    pre_train_dataset = "./train_data/news_crawl/news2012_len_32.tfrecord-001-of-001"
    fine_tune_dataset = "./train_data/ggw_data/gigaword_train_dataset.tfrecord-001-of-001"
    test_dataset = "./train_data/ggw_data/gigaword_test_dataset.tfrecord-001-of-001"
    dataset_sink_mode = False
    dataset_sink_step = 100
    random_seed = 100
    save_graphs = False
    seq_length = 32 #64
    vocab_size = 39284
    hidden_size = 1024
    num_hidden_layers = 6
    num_attention_heads = 8
    intermediate_size = 4096
    hidden_act = "relu"
    hidden_dropout_prob = 0.2
    attention_dropout_prob = 0.2
    max_position_embeddings = 32 #64
    initializer_range = 0.02
    label_smoothing = 0.1
    beam_width = 4
    length_penalty_weight = 1.0
    max_decode_length = 32 #64
    init_loss_scale = 65536
    loss_scale_factor = 2
    scale_window = 200
    lr = 0.0001
    poly_lr_scheduler_power = 0.5
    decay_steps = 10000
    decay_start_step = 12000
    warmup_steps = 4000
    min_lr = 0.000001
    save_ckpt_steps = 10000
    keep_ckpt_max = 50
    ckpt_prefix = "pt"
    metric = "rouge"
    vocab = "./vocab/all_en.dict.bin"
    output = "./output/infer.bin"

In [ ]:
import os
import pickle
import numpy as np

from mindspore.common.tensor import Tensor
from mindspore.nn import Momentum
from mindspore.nn.optim import Adam, Lamb
from mindspore.train.model import Model
from mindspore.train.loss_scale_manager import DynamicLossScaleManager, FixedLossScaleManager
from mindspore.train.callback import CheckpointConfig, ModelCheckpoint, TimeMonitor
from mindspore import context, Parameter
from mindspore.communication import management as MultiAscend
from mindspore.train.serialization import load_checkpoint
from mindspore.common import set_seed

from src.dataset import load_dataset
from src.transformer import TransformerNetworkWithLoss, TransformerTrainOneStepWithLossScaleCell
from src.utils import LossCallBack
from src.utils import one_weight, zero_weight, weight_variable
from src.utils import square_root_schedule
from src.utils.lr_scheduler import polynomial_decay_scheduler, BertLearningRate

In [ ]:
config = config()
config.epochs = 1 #for test

print(" | Starting training on single device.")
pre_train_dataset = load_dataset(data_files=config.pre_train_dataset,
                                 batch_size=config.batch_size,
                                 epoch_count=1,
                                 sink_mode=config.dataset_sink_mode,
                                 sink_step=config.dataset_sink_step)

# 定义帶loss的網路
net_with_loss = TransformerNetworkWithLoss(config, is_training=True)
net_with_loss.init_parameters_data()

for param in net_with_loss.trainable_params():
    name = param.name
    value = param.data
    if isinstance(value, Tensor):
        if name.endswith(".gamma"):
            param.set_data(one_weight(value.asnumpy().shape))
        elif name.endswith(".beta") or name.endswith(".bias"):
            param.set_data(zero_weight(value.asnumpy().shape))
        else:
            param.set_data(weight_variable(value.asnumpy().shape))

update_steps = config.epochs * pre_train_dataset.get_dataset_size()

# 定义递减的学习率
lr = Tensor(polynomial_decay_scheduler(lr=config.lr,
           min_lr=config.min_lr,
           decay_steps=config.decay_steps,
           total_update_num=update_steps,
           warmup_steps=config.warmup_steps,
           power=config.poly_lr_scheduler_power), dtype=mstype.float32)
# 定义优化器
optimizer = Adam(net_with_loss.trainable_params(), lr, beta1=0.9, beta2=0.98)

# loss scale (mode = dynamic)
scale_manager = DynamicLossScaleManager(init_loss_scale=config.init_loss_scale,
                                        scale_factor=config.loss_scale_factor,
                                        scale_window=config.scale_window)
# 定义反向网络
net_with_grads = TransformerTrainOneStepWithLossScaleCell(network=net_with_loss, optimizer=optimizer,
                                                          scale_update_cell=scale_manager.get_update_cell())
net_with_grads.set_train(True)

# 初始化模型
model = Model(net_with_grads)

time_cb = TimeMonitor(data_size=pre_train_dataset.get_dataset_size())
ckpt_config = CheckpointConfig(save_checkpoint_steps=config.save_ckpt_steps,
                               keep_checkpoint_max=config.keep_ckpt_max)

callbacks = []
callbacks.append(time_cb)
ckpt_save_dir = config.save_checkpoint_path

ckpt_callback = ModelCheckpoint(
    prefix=config.ckpt_prefix,
    directory=os.path.join(ckpt_save_dir, 'ckpt_{}'.format(os.getenv('DEVICE_ID'))),
    config=ckpt_config)
loss_monitor = LossCallBack(rank_id=os.getenv('DEVICE_ID'))
callbacks.append(loss_monitor)
callbacks.append(ckpt_callback)

print(" | Start pre-training job.")
model.train(config.epochs, pre_train_dataset,
            callbacks=callbacks, dataset_sink_mode=config.dataset_sink_mode,
            sink_size=config.dataset_sink_step)

## 3.5 微调

微调时，根据不同的任务，采用不同的数据集对预训练的模型进行微调。



In [ ]:
config = config()
config.checkpoint_file_path = './output/checkpoint/ckpt_0/pt-21_80860.ckpt' #赋值给预训练生成的已有模型文件
config.ckpt_prefix = "ft"
config.epochs = 1 #for test

print(" | Starting training on single device.")
fine_tune_dataset = load_dataset(data_files=config.fine_tune_dataset,
                                 batch_size=config.batch_size,
                                 epoch_count=1,
                                 sink_mode=config.dataset_sink_mode,
                                 sink_step=config.dataset_sink_step)

# 定义帶loss的網路
net_with_loss = TransformerNetworkWithLoss(config, is_training=True)
net_with_loss.init_parameters_data()

# 读取已有模型文件的权重
weights = load_checkpoint(config.checkpoint_file_path)
for param in net_with_loss.trainable_params():
    weights_name = param.name
    if isinstance(weights[weights_name], Parameter):
        param.set_data(weights[weights_name].data)
    elif isinstance(weights[weights_name], Tensor):
        param.set_data(Tensor(weights[weights_name].asnumpy(), config.dtype))
    elif isinstance(weights[weights_name], np.ndarray):
        param.set_data(Tensor(weights[weights_name], config.dtype))
    else:
        param.set_data(weights[weights_name])

update_steps = config.epochs * fine_tune_dataset.get_dataset_size()

# 定义递减的学习率
lr = Tensor(polynomial_decay_scheduler(lr=config.lr,
           min_lr=config.min_lr,
           decay_steps=config.decay_steps,
           total_update_num=update_steps,
           warmup_steps=config.warmup_steps,
           power=config.poly_lr_scheduler_power), dtype=mstype.float32)
# 定义优化器
optimizer = Adam(net_with_loss.trainable_params(), lr, beta1=0.9, beta2=0.98)

# loss scale (mode = dynamic)
scale_manager = DynamicLossScaleManager(init_loss_scale=config.init_loss_scale,
                                        scale_factor=config.loss_scale_factor,
                                        scale_window=config.scale_window)
# 定义反向网络
net_with_grads = TransformerTrainOneStepWithLossScaleCell(network=net_with_loss, optimizer=optimizer,
                                                          scale_update_cell=scale_manager.get_update_cell())
net_with_grads.set_train(True)

# 初始化模型
model = Model(net_with_grads)

time_cb = TimeMonitor(data_size=fine_tune_dataset.get_dataset_size())
ckpt_config = CheckpointConfig(save_checkpoint_steps=config.save_ckpt_steps,
                               keep_checkpoint_max=config.keep_ckpt_max)

callbacks = []
callbacks.append(time_cb)
ckpt_save_dir = config.save_checkpoint_path

ckpt_callback = ModelCheckpoint(
    prefix=config.ckpt_prefix,
    directory=os.path.join(ckpt_save_dir, 'ckpt_{}'.format(os.getenv('DEVICE_ID'))),
    config=ckpt_config)
loss_monitor = LossCallBack(rank_id=os.getenv('DEVICE_ID'))
callbacks.append(loss_monitor)
callbacks.append(ckpt_callback)

print(" | Start fine-tuning job.")
model.train(config.epochs, fine_tune_dataset,
            callbacks=callbacks, dataset_sink_mode=config.dataset_sink_mode,
            sink_size=config.dataset_sink_step)

## 3.6 推理
测试过程中，通过微调后的模型预测结果，并采用波束大小為4的搜索算法获取可能性最高的预测结果。


In [ ]:
import os
import pickle
import time

import mindspore.nn as nn
import mindspore.common.dtype as mstype
from mindspore.ops import operations as P
from mindspore.common.tensor import Tensor
from mindspore.train.model import Model
from mindspore.train.serialization import load_checkpoint, load_param_into_net

from src.utils import Dictionary
from src.dataset import load_dataset
from src.transformer.transformer_for_infer import TransformerInferModel
from src.transformer.transformer_for_train import TransformerTraining
from src.utils.load_weights import load_infer_weights
from src.utils.rouge_score import rouge

class TransformerInferCell(nn.Cell):
    def __init__(self, network):
        super(TransformerInferCell, self).__init__(auto_prefix=False)
        self.network = network

    def construct(self,
                  source_ids,
                  source_mask):
        predicted_ids, predicted_probs = self.network(source_ids,
                                                      source_mask)
        return predicted_ids, predicted_probs

def get_rouge_score(result, vocab):
    """Calculate ROUGE score."""
    predictions = []
    targets = []
    for sample in result:
        predictions.append(' '.join([vocab[t] for t in sample['prediction']]))
        targets.append(' '.join([vocab[t] for t in sample['target']]))
        print(f" | source: {' '.join([vocab[t] for t in sample['source']])}")
        print(f" | prediction: {predictions[-1]}")
        print(f" | target: {targets[-1]}")

    return rouge(predictions, targets)

config = config()
config.epoch = 1 #for test
config.checkpoint_file_path = "./output/checkpoint/ckpt_0/ft_2-1_10319.ckpt"

vocab = Dictionary.load_from_persisted_dict(config.vocab)

eval_dataset = load_dataset(data_files=config.test_dataset,
                            batch_size=config.batch_size,
                            epoch_count=1,
                            sink_mode=config.dataset_sink_mode,
                            shuffle=False)

tfm_model = TransformerInferModel(config=config, use_one_hot_embeddings=False)
tfm_model.init_parameters_data()

params = tfm_model.trainable_params()
weights = load_infer_weights(config)

for param in params:
    value = param.data
    name = param.name
    with open("weight_after_deal.txt", "a+") as f:
        weights_name = name
        f.write(weights_name + "\n")
        if isinstance(value, Tensor):
            if weights_name in weights:
                assert weights_name in weights
                param.set_data(Tensor(weights[weights_name], mstype.float32))
            else:
                raise ValueError(f"{weights_name} is not found in checkpoint.")
        else:
            raise TypeError(f"Type of {weights_name} is not Tensor.")

print(" | Load weights successfully.")

tfm_infer = TransformerInferCell(tfm_model)
model = Model(tfm_infer)

predictions = []
probs = []
source_sentences = []
target_sentences = []
for batch in eval_dataset.create_dict_iterator(output_numpy=True, num_epochs=1):
    source_sentences.append(batch["source_eos_ids"])
    target_sentences.append(batch["target_eos_ids"])

    source_ids = Tensor(batch["source_eos_ids"], mstype.int32)
    source_mask = Tensor(batch["source_eos_mask"], mstype.int32)

    start_time = time.time()
    predicted_ids, entire_probs = model.predict(source_ids, source_mask)
    print(f" | Batch size: {config.batch_size}, "
          f"Time cost: {time.time() - start_time}.")

    predictions.append(predicted_ids.asnumpy())
    probs.append(entire_probs.asnumpy())

output = []
for inputs, ref, batch_out, batch_probs in zip(source_sentences,
                                               target_sentences,
                                               predictions,
                                               probs):
    for i in range(config.batch_size):
        if batch_out.ndim == 3:
            batch_out = batch_out[:, 0]

        example = {
            "source": inputs[i].tolist(),
            "target": ref[i].tolist(),
            "prediction": batch_out[i].tolist(),
            "prediction_prob": batch_probs[i].tolist()
        }
        output.append(example)

with open(config.output, "wb") as f:
    pickle.dump(output, f, 1)

score = get_rouge_score(output, vocab)
print(score)

## 3.7 评估指标

### 3.7.1 PPL
对于自然语言生成模型，最重要的问题就是生成的文本序列是否符合我们人类的使用习惯。
在自然语言处理领域中，最常用的评估指标为PPL，perplexity（困惑度），它可以衡量语言模型的好坏，计算方法是根据每个词来估计一句话出现的概率，并用句子长度作标准化。

$$
PPL(S) = P(w_1w_2...w_N)^{-\frac{1}{N}} \\
 = \sqrt[N]{\frac{1}{p(w_1 w_2...w_N)}} \\
 = \sqrt[N]{\Pi^N_{i=1} \frac{1}{p(w_i | w_1w_2...w_{i-1})}}
$$

其中$S$代表输入的sentence，$N$为句子长度，$p(w_i)$是第$i$个词的概率。第一个词就是 $p(w_1|w_0)$，而$w_0$表示句子的起始，是个占位符。

PPL越小，$p(w_i)$则越大，我们期望的sentence出现的概率就越高。Perplexity可以认为是average branch factor（平均分支系数），即预测下一个词时可以有多少个合理的选择，可选词数越少，表示生成的句子越接近目标，模型越准确。但perplexity只是大致估计下训练效果，它不是完全意义上的标准，具体问题还是要具体分析。

### 3.7.2 ROUGE
ROUGE指标是在机器翻译、自动摘要、问答生成等领域常见的评估指标，全称是 (Recall-Oriented Understudy for Gisting Evaluation)。ROUGE通过将模型生成的摘要或者回答与参考答案（一般是人工生成的）进行比较计算，得到对应的得分。ROUGE指标与BLEU指标非常类似，均可用来衡量生成结果和标准结果的匹配程度，不同的是ROUGE基于召回率，BLEU更看重准确率。

在文本摘要微调任务上，使用 Gigaword 测试集计算ROUGE-1、ROUGE-2 和 ROUGE-L 的 F1 分数作为微调模型的评估结果。

ROUGE-1和ROUGE-2属于ROUGE-N算法，就是将模型生成的结果和标准结果按N-gram拆分后，计算召回率。公式为：
$$
ROUGE-N = \frac{\sum_{S \in ReferenceSummaries} \sum_{gram_n \in S} Count_{match}gram_n}{\sum_{S \in ReferenceSummaries} \sum_{gram_n \in S}Count(gram_n)}
$$

分子表示所有样本按N-gram拆分后与生成的结果按N-gram拆分后匹配上个数的和；分母表示所有样本的标准结果，按N-gram拆分后的和。

Rouge-L的L意为: Longest Common Subsequence，最长公共子序列。Rouge-L的公式可以表示为：
$$
R_{lcs} = \frac{LCS(X, Y)}{m}\\
P_{lcs} = \frac{LCS(X, Y)}{n}\\
F_{lcs} = \frac{(1+\beta^2)R_{lcs}P_{lcs}}{R_{lcs}+\beta^2 P_{lcs}}
$$

公式中的$F_{lcs}$就是ROUGE-L的得分。


